In [68]:
import numpy as np
from sklearn.cluster import DBSCAN
import pandas as pd
import geopandas as gpd
from shapely import wkt
import matplotlib as plt
import requests
from urllib.parse import quote
import pyproj
import os
os.chdir("D:/Projects/crime-prediction")
pyproj.datadir.set_data_dir("D:/ProgramData/anaconda3/envs/crime-prediction/Library/share/proj")

In [ ]:
pre_processed_file_path = 'data/preprocessed/preprocessed-1.csv'
city_limits_path = 'data/preprocessed/city-limits.csv'
weather_apikey = 'MQK9MND4NLXX6MGYRAWDVZX6E'

In [ ]:
def parse_preprocessed_data_1():
    df = pd.read_csv(
                         pre_processed_file_path,
                         parse_dates=['Offense Date'],                       
                         dtype={
                             'Precinct': 'category',
                             'Offense Category': 'category',
                             'Latitude': np.float32,
                             'Longitude': np.float32
                         }
                        )
    gdf = gpd.GeoDataFrame(df, geometry= df["geometry"].apply(wkt.loads), crs='EPSG:4326')
    return gdf

In [ ]:
def parse_citylimits():
    df = pd.read_csv(city_limits_path)
    df["geometry"] = df["geometry"].apply(wkt.loads)
    df["centroid"] = gpd.GeoSeries.from_wkt(df['centroid'])
    gdf = gpd.GeoDataFrame(df, geometry= 'geometry', crs='EPSG:4326')    
    return gdf

In [75]:
def get_weather_visualcrossing(lat, long, date_time, api_key):
    location = quote(f"{long},{lat}")
    base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
    url =f"{base_url}/{location}/{date_time}?unitGroup=metric&include=hours&key={api_key}&contentType=json"
    print(url)
    response = requests.get(url)
    print(response.text)
    return response.json()

In [ ]:
gdf = parse_preprocessed_data_1()

In [ ]:
city_limits_gdf = parse_citylimits()

In [ ]:
gdf['Offense Date Rounded Hour'] = gdf['Offense Date'].dt.round('h')
unique_combinations = gdf[['Latitude', 'Longitude','Offense Date Rounded Hour']].drop_duplicates()
grouped_df = unique_combinations.groupby('Offense Date Rounded Hour').agg('count')

In [ ]:
cf.to_csv('data/preprocessed/preprocessed-2.csv', index=False)

In [ ]:
group_counts =  grouped_df['Latitude'].size().to_frame().sort_values(by='Latitude', ascending=False)
print(group_counts[group_counts['Latitude']>1])

In [76]:
weather_data = get_weather_visualcrossing(t[0].x, t[0].y, "2021-12-22T05:00:00", weather_apikey)
print(weather_data)



https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/47.62048706590659%2C-122.33667448624294/2021-12-22T05:00:00?unitGroup=metric&include=hours&key=MQK9MND4NLXX6MGYRAWDVZX6E&contentType=json
{"queryCost":24,"latitude":47.62048706590659,"longitude":-122.33667448624294,"resolvedAddress":"47.62048706590659,-122.33667448624294","address":"47.62048706590659,-122.33667448624294","timezone":"America/Los_Angeles","tzoffset":-8.0,"days":[{"datetime":"2021-12-22","datetimeEpoch":1640160000,"tempmax":9.9,"tempmin":4.8,"temp":7.9,"feelslikemax":7.8,"feelslikemin":2.0,"feelslike":5.3,"dew":6.0,"humidity":87.9,"precip":12.191,"precipprob":100.0,"precipcover":58.33,"preciptype":["rain"],"snow":0.0,"snowdepth":0.0,"windgust":42.5,"windspeed":23.4,"winddir":157.3,"pressure":1003.3,"cloudcover":96.2,"visibility":13.3,"solarradiation":49.4,"solarenergy":4.3,"uvindex":1.0,"sunrise":"07:55:35","sunriseEpoch":1640188535,"sunset":"16:20:53","sunsetEpoch":1640218853,"moonphase":